In [12]:
# Libraries 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from tqdm import tqdm
import torch

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split  
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import TextClassificationPipeline

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

import random
import string 
import re
import tweepy as tw
import kafka
import json

import nltk
from nltk.corpus import stopwords

import tweepy as tw

%matplotlib inline

In [14]:
#Create word contractions mapper
acrynoms_dic = {"omg":"oh my god", "wth":"what the hell", "asap":"as soon as possible", "rip":"rest in peace",
                "fomo":"fear of missing out","yolo":"you only live once","awol":"absent without leave",
                "covid":"corona virus disease","icu":"intensive care unit", "sars":"severe acute respiratory syndrome",
                "postop":"post operative", "preop":"pre operative","btw":"by the way"}
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower() #Convert text to lowercase
    text = re.sub('@[^ ]+','',text) #Remove Username handle
    text = re.sub('#[^ ]+','',text) #Remove Hashtags
    text = re.sub(r'^rt ', '', text) #Remove RT (Retweets) tag
    text = re.sub('\[.*?\]', '', text) #Remove and brackets
    text = re.sub('https?://\S+|www\.\S+', '', text) #Remove links
    text = re.sub('<.*?>+', '', text) #Remove Special Characters
    neg_pattern = re.compile(r'\b(' + '|'.join(acrynoms_dic.keys()) + r')\b')
    neg_handled = neg_pattern.sub(lambda x: acrynoms_dic[x.group()], text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) #Remove Punctuation
    text = re.sub('\n', '', neg_handled) #Remove line breaks
    text = re.sub('\w*\d\w*', '', text) #Remove words containing numbers
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text) #Remove single characters
    text = re.sub(r'\s+', ' ', text) #Remove double spacing
    return text


def text_preprocessing(text):
    """
    Cleaning, tokenizing, removing stopwords and parsing the text.

    """
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    nopunc = clean_text(text)
    tokenized_text = tokenizer.tokenize(nopunc)
    remove_stopwords = [w for w in tokenized_text if w not in stopwords.words('english')]
    combined_text = ' '.join(remove_stopwords)
    return combined_text

In [7]:
model=torch.load('Final/Final_BERT_model.pth',map_location=torch.device('cpu'))
model.config

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.3,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.3,
  "hidden_size": 768,
  "id2label": {
    "0": "Negative",
    "1": "Neutral",
    "2": "Positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case = True)

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer)

### Accessing Twitter API

In [9]:
consumer_key="urw8uTZiLJpDF1yJbI8woho43"
consumer_secret="xA2eVaDtNOtRm9NY0G33VhU2JoQSF3HVBWAZIAdgbRsvEZwI6u"
access_token="350057086-dz0cUQruL3X7U22IqFuO0UWm1NOlVzInsCJ1dV1M"
access_token_secret="vX49zHJ90Y0kW4oVADr0GQXNlf65KbtDf5IjEsLuPg2dr"

In [10]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

### Requesting Tweets

In [19]:
keysearch=input("Enter Tag: ")
search_tweets = api.search_tweets(keysearch,count=300,tweet_mode='extended',lang='en')
tweets=[]

for tweet in search_tweets:
    if 'retweeted_status' in tweet._json:
        tweet_text = tweet._json['retweeted_status']['full_text']
        tweets.append(tweet_text)
    else:
        tweet_text = tweet.full_text
        tweets.append(tweet_text)
           
data_twitter = pd.DataFrame(tweets, columns=['Tweet'])
data_twitter

Enter Tag: Covid 19


,Tweet
0,BREAKING: Fox's Peter Doocy returns to White H...
1,Hella jealous of all my tattoo mutuals just SL...
2,"Today we have recorded 20,566 new COVID-19 cas..."
3,end of a 12hour day in practice....colleagues ...
4,Hong Kong will start offering COVID-19 vaccine...
...,...
95,India began administering Precaution Doses. Ku...
96,https://t.co/79MLc8CoS8\nThis is the Met Polic...
97,Record numbers in #Spain today - and this plan...
98,‘SHAMEFUL’\n\nLOOK: Former beauty queen Maggie...


## Making predictions

#### Batch predictions

In [32]:
def batch_pred(data_twitter):
    clean=[]
    out_sentiment=[]
    out_confidence=[]
    for tweet in data_twitter.Tweet:
        clean_tweet=clean_text(tweet)
        pred=pipe(clean_tweet)
        clean.append(clean_tweet)
        out_sentiment.append(pred[0]['label'])
        out_confidence.append(pred[0]['score'])
    data_twitter['Clean tweet'] = pd.DataFrame(clean)
    data_twitter['Sentiment'] = pd.DataFrame(out_sentiment)
    data_twitter['Confidence'] = pd.DataFrame(out_confidence) 

    return data_twitter

In [39]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth',None)
pd.set_option('display.width', 2000)

In [44]:
data_twitter=batch_pred(data_twitter)

In [45]:
data_twitter

,Tweet,Clean tweet,Sentiment,Confidence
0,Pfizer now admits boosters are only 40-50% effective on Covid and only for 10 wks and two doses don’t help w/Omicron.But some leaders still want to force people?In light of this science when does forcing people become a criminal act or at least malpractice https://t.co/Gsqfbb53vh,pfizer now admits boosters are only -% effective on corona virus disease and only for wks and two doses don’t help w/omicron.but some leaders still want to force people?in light of this science when does forcing people become criminal act or at least malpractice,Negative,0.730159
1,"This doesn't sound right. Covid is a US-made recombinant bat vaccine? Fauci continued the DARPA rejected project? How does Joseph Murphy purport to know all these things? \n\nMy guess is he's conflating stuff. If he isn't, this would be the smoking gun of all smoking guns. https://t.co/8e6uJm69bl https://t.co/wpKYfcOSO1","this doesn't sound right. corona virus disease is us-made recombinant bat vaccine? fauci continued the darpa rejected project? how does joseph murphy purport to know all these things? my guess is he's conflating stuff. if he isn't, this would be the smoking gun of all smoking guns.",Negative,0.938731
2,SCIENTISTS: We developed a vaccine which makes you *20X less likely* to die of Covid.\n\nREPUBLICANS: What if we drank pee instead?,scientists: we developed vaccine which makes you * less likely* to die of corona virus disease.republicans: what if we drank pee instead?,Negative,0.980986
3,"California just reported an astonishing 308,820 new cases today, meaning cases are up 1,774% in only a few weeks since the statewide mask mandate was reinstated\n\nI wonder if anyone will ask Gavin Newsom if he regrets pretending that mask and vaccine mandates would stop COVID https://t.co/jrkG5QHvjt","california just reported an astonishing , new cases today, meaning cases are up ,% in only few weeks since the statewide mask mandate was reinstatedi wonder if anyone will ask gavin newsom if he regrets pretending that mask and vaccine mandates would stop corona virus disease",Neutral,0.511293
4,"Facts! Myocarditis a very rare side effect of Pfizer &amp; Moderna, usually temporary, most better within days. Myocarditis 1-2 per 100,000 people who receive Pfizer (in young men &amp; teenage boys after 2nd dose 8-12 per 100,000). My teenage son is vaccinated. https://t.co/XLcwJxEPAl","facts! myocarditis very rare side effect of pfizer &amp; moderna, usually temporary, most better within days. myocarditis - per , people who receive pfizer (in young men &amp; teenage boys after dose - per ,). my teenage son is vaccinated.",Positive,0.546012
5,Pfizer now admits boosters are only 40-50% effective on Covid and only for 10 wks and two doses don’t help w/Omicron.But some leaders still want to force people?In light of this science when does forcing people become a criminal act or at least malpractice https://t.co/Gsqfbb53vh,pfizer now admits boosters are only -% effective on corona virus disease and only for wks and two doses don’t help w/omicron.but some leaders still want to force people?in light of this science when does forcing people become criminal act or at least malpractice,Negative,0.730159
6,"Rep @RepMTG let's Twit and FB have it for pushing her off the websites. Also, goes after what really happened on Jan 6th, asks why we can't get all the evidence and outlines what the commission is trying to accomplish. Must see interview! https://t.co/KEuQOMYNjT","rep let's twit and fb have it for pushing her off the websites. also, goes after what really happened on jan , asks why we can't get all the evidence and outlines what the commission is trying to accomplish. must see interview!",Neutral,0.851023
7,"Does your child 12-15 need a COVID-19 booster? They can now get one! The boosters were authorized earlier this week by the FDA and CDC, and on January 6 were authorized by the Western States Scientific Review Committee. See our vaccine webpage

#### Stream predictions

### Kafka Streaming and predictions

In [17]:
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

In [18]:
class MyListener(tw.Stream):
    def on_status(self, data):
        text= clean_text(data.text)
        pred= pipe(text)
        tweet_dict = {
            "text": data.text,
            "clean_text": text,
            "Sentiment": pred[0]['label'],
            "Confidence": pred[0]['score'],
            "user_name": data.user.name,
            "screen_name": data.user.screen_name,
            "id_string": data.user.id_str,
            "location": data.user.location,
            "lang" : data.user.lang == 'en',
        }
        print(data.text +" \nPredication:",pred[0]['label'])
        #THE FOLLOWING LINE SENDS DATA IN KAFKA (Under topic "trump").
        producer.send("covid", json.dumps(tweet_dict).encode("utf-8"))
        return True
    def on_error(self, status_code):
        if status_code == 420:
            return False

producer = kafka.KafkaProducer(bootstrap_servers='localhost:9092')

twitter_stream = MyListener(
    consumer_key, consumer_secret,
    access_token, access_token_secret)
twitter_stream.filter(track=["covid"])

RT @hparadoxa: Je réalise qu'il y a un truc pas clair pour pas mal de gens : Le Covid Long, c'est pas le Covid pendant longtemps.

Les gens… 
Predication: Neutral
RT @JoshuaPotash: holy shit the U.S. reported more than 1.4 million new covid cases today. i don't see how we're not paying people to stay… 
Predication: Negative
RT @AlertaNews24: 🇨🇳 | AHORA - CORONAVIRUS: La OMS dice que las medidas estrictas de los Juegos Olímpicos de Beijing para combatir la propa… 
Predication: Neutral
RT @TheLeoTerrell: Attention Doctors: Unlawful to prioritize Covid-19 treatment based upon the race of your patient.  YOU HAVE BEEN WARNED! 
Predication: Negative
No se si soy positivo en covid, embarazo o es el golpe de calor. Pero que la estoy sufriendo, la estoy sufriendo lpm 😓 
Predication: Positive
RT @CNN: "For many Australians, the problem wasn't about Djokovic's paperwork -- it was whether he considered himself above the country's p… 
Predication: Neutral
RT @JoshuaPotash: holy shit the U.S. report

RT @montse_muse_um: Plena ola de Omicron y a mi mamá le dijeron que a huevo tiene que llevar al IMSS a mis dos abuelos (uno en silla de rue… 
Predication: Neutral
Things I did in my White-T pre-covid: 
✅Slang 
✅Bang 
✅All in the club spitting game  

Things I do in my white-t a… https://t.co/L2Uz2h2X2X 
Predication: Neutral
拜託疫情不要毀了我今年所有的計畫… 
Predication: Neutral
RT @LeMediaTV: 🔴Direct🔴ÉCOLE SOUS COVID : L'EDUCATION "RÉPOND" À CASTEX

https://t.co/4QkTdUkgLq

#Grèvegénérale de l'#EducationNationale :… 
Predication: Neutral
Nunca le han interesado. 
Predication: Neutral
https://t.co/20toRcw8i3 
Predication: Neutral
RT @jzux: my friend is dating a guy who works for Google and i kid you not apparently they gave him a little high-tech at-home COVID test m… 
Predication: Positive
RT @MurphyWoodhouse: WOW. Check out this week-to-week #Covid case count visualization for #Sonora, from @dogomoreno. Behold the power of #O… 
Predication: Positive
RT @Blue_Footy: Chelsea have lost just one match s

RT @ChloeMurder: @jvipondmd Hi Dr. Vipond, can you add healthcare workers with active Covid to your analysis? We went from 3982 on Friday t… 
Predication: Positive
RT @bobequalbobbkwd: My mother-in-law (85, Parkinson’s) caught Covid from her unvaxxed caregiver and is now in the ICU and may not survive.… 
Predication: Neutral
RT @j_thePA: Is burnout hoping your covid test is positive so you can have 5 days off 
Predication: Positive
RT @Notyour28981739: The CARES Act gives  $0  for early treatment... 
Predication: Positive
RT @grahamwalker: “Airborne” is a hard concept to grasp: The fact you can get Covid from someone breathing the same air — but who left the… 
Predication: Neutral
RT @_titouncle: NO I DONT CARE IF ITS MILD I DO NOT WANT TO CATCH COVID IS THAT SO HARD TO UNDERSTAND NOWADAYS?!?!?!?! 
Predication: Negative
RT @MBBharath001: Akhira and Renu Desai Tested Positive For Covid 19

Wishing them a speedy recovery 🙏GetWellSoon ❤️ TakeCare https://t.co/… 
Predication: Positive
RT @

RT @LocationUnkn0wn: อิเหี้ย ไปๆมาๆสถานการณ์แย่กว่าเดิม โนวัคมีเอกสารยืนยันว่าติดโควิดวันที่ 16 ธันวา (จะได้เข้าออสได้โดยไม่ต้องฉีดวัคซีน)… 
Predication: Neutral
@LisaMarieBoothe Excess deaths on a yearly average increased by 700,000, people died, it quickened their deaths whe… https://t.co/GSU3JeqrEv 
Predication: Negative
RT @Harjind51464017: To fight covid-19, our guru @Gurmeetramrahim ji has added two more new works for the betterment of humanity, that you… 
Predication: Neutral
RT @NormandieMemes: Deux ans de Covid, c'est aussi deux ans sans entendre des touristes Américains prononcer "Keuhan", "Carhantan", "Baayeu… 
Predication: Neutral
End mass jabs and live with Covid, says ex-head of vaccine taskforce https://t.co/eGjhmIS6co 
Predication: Neutral
RT @MIKITO_777: COVID-19は肺炎が悪化して酸欠になっているのに息苦しさを感じない、
happy hypoxiaという症状が出ることがあります。

なのでパルスオキシメーターで血中酸素飽和度が下がっているのを確認して重症化がはじめて分かることがあります。… 
Predication: Positive
RT @GerryCacanindin: Robredo’s Bayanihan E-Konsulta sends free COVID car

Predication: Neutral
RT @neuroecology: This post from a NYC high schooler about the covid situation right now is wild https://t.co/iefprZq2fy https://t.co/2xSIW… 
Predication: Neutral
BinaxAtSomePointThisCentury. #BinaxNow #COVIDTesting 
Predication: Neutral
RT @pearlsintea: korang masak apa? i masak covid https://t.co/L07e3i9wvp 
Predication: Neutral
RT @Excelsior: 🔴☣ En su mensaje, la jefa de Gobierno, @Claudiashein no informó si se aislará o realizará una prueba de detección de #covid,… 
Predication: Neutral
RT @sneerid: covid please stop ✋ NOW! 
Predication: Neutral
RT @TomFitton: Fauci knew. 
Predication: Neutral
RT @guillermofesser: No se  puede meter en el mismo saco a una señora, la hermana Greene,que no ha presentado una sola proposición de ley y… 
Predication: Neutral
RT @DariuszSzumilo: Albo się zgadzasz z Radą Medyczną „że szczepienie przeciw COVID-19 jest procedurą medyczną ratująca zdrowie i życie” al… 
Predication: Neutral
RT @hansvantelling: Engeland en Spanje verklaren

...

RT @AGRICOMMODITIE1: @SaiKate108 The Heart Muscles never regenerate … Graphene Hydroxides are like microscopic razor blades … spike Protein… 
Predication: Positive
RT @ifyouseekkae: Is it just common colds or is it covid? The only way to know is to pay out around 3k, which most people would rather use… 
Predication: Neutral
me: hey yeah i have covid symptoms 
teacher: okay, see you later in school! 
Predication: Positive
RT @USBornNRaised: Yup. You were duped.

The CDC has said from jump only 6% of the total deaths are from Covid only.

6% of 800,000 is 48,0… 
Predication: Neutral
RT @PawanCharanFC: #AkiraNandan &amp; #RenuDesai garu had Tested COVID Positive. Wishing Both a very speedy Recovery! 

Everyone Please Wear Ma… 
Predication: Positive
RT @Reezyard: Wat gebeurt hier? Zelfs op CNN!

Na twee jaar gaat men eindelijk wat duidelijker zijn in de rapportage van ziekenhuisopnames.… 
Predication: Neutral
@Rantxo1 @gorka_orive Y entonces si has estado en la UCI x Covid, y tienes secuel

RT @JackPosobiec: They won't tell us how many people have died of COVID without any comorbidities 
Predication: Negative
RT @carolineboudet: J'ai l'intuition que la prochaine mesure pour limiter la propagation du covid sera de ne plus communiquer aucun chiffre… 
Predication: Neutral
@shane_united @NancyJoBaker @JustinTrudeau Vaccines are a ‘cure’ or going to ‘stop’ covid. As a fully vaccinated pe… https://t.co/NNGZSWs4Ku 
Predication: Neutral
RT @dgardner: "I was recently in Nashville, Tennessee," writes @jordanbpeterson. "No lockdowns. No masks. No COVID regulations to speak of.… 
Predication: Neutral
RT @redrumthewitch: s/o to the thick layer of resin covering my lungs and protecting me from covid 
Predication: Neutral
Fakültede her gün bir ya da iki çalışma arkadaşımızın covid+ olduğunu öğreniyoruz. Öğrencilerimizin hangisi temaslı… https://t.co/EbZSDhQLAh 
Predication: Neutral
RT @MSEQUEYRO: Cobarde, racista, miserable, clasista. Este señor no puede ser contratado en ninguna empres

RT @GariGurman: This week China covid tested a city the size of Chicago &amp; New York City combined in just six hours. In comparison, people i… 
Predication: Neutral
@babbaranu @ArvindKejriwal सड़जी कहां से लाते हो इतनी मक्कारी?
#kejriwal #covid #isolation #aamaadmiparty https://t.co/IN373sbplH 
Predication: Neutral
RT @drkodnani: #PMCBankCrisis 
Banking sector India needs to no corrupt, efficient,who have knowledge of economics.India is vast country wi… 
Predication: Neutral
@lynnewalker5492 @Hayakuishi @MeMikeD1 @Travis_in_Flint "For example, the CDC in the USA has stated that it counts… https://t.co/5YG7LeltpT 
Predication: Neutral
No covid, no vaccines, but having weird neck stiffness and 4 day streak of headache. Anyway life goes on and I aint… https://t.co/EKi6pgDx36 
Predication: Neutral
Chicago teachers union strike deal on COVID safety with Mayor Lightfoot that will see schools reopen - https://t.co/0XA5o8fiCZ 
Predication: Neutral
RT @tamboretedforro: Negativo pra COVID e po

I support anyone who stands for truth. 
#covid is a #scam 
#covidscam
#plandemic
#Scamdemic 
Predication: Positive
RT @Balagadde5: The Minister of Health, @JaneRuth_Aceng  will this Friday address the Nation on the COVID-19 response and Vaccination cover… 
Predication: Neutral
RT @lgnacioduh: besame, no tengo covid 
Predication: Neutral
@DocJeffD do we have a plan for King County? 
Predication: Neutral
Real-World Experience with COVID-19, Including Direct COVID-19 Antigen Testing and Monoclonal-Antibody Bamlanivimab… https://t.co/wFleRNFLUT 
Predication: Neutral
No media outlet reported this, nor the protests in France Germany Australia. Where the hell is real true media? Con… https://t.co/KFK7ISuDQd 
Predication: Negative
RT @follforfight: วันก่อนคุยกับคนติดโควิดแล้วหาย หลังโควิดมาหลายเดือนก็ยังมีอาการเหนื่อยง่ายขึ้น นอนไม่หล้บ ปวดข้อ ไอเรื่อยๆ (เรียก Long co… 
Predication: Neutral
LIST: More Cincinnati-area schools going remote amid COVID-19 surge https://t.co/0oT5JjyvGk 
Predication

KeyboardInterrupt: 